In [1]:
import boto3
import glob
import tqdm

In [2]:
s3_resource = boto3.resource(service_name='s3',
                             region_name='ru-central1',
                             endpoint_url='https://storage.yandexcloud.net',
                             aws_access_key_id='QM2SjYCUKftEAz4VSCcW',
                             aws_secret_access_key='hmgmfAq8hLXsY3KInSuiud2JUQcuULfoqB3lkMj9')

In [3]:
a = s3_resource.Bucket(name='issoproduction') 

In [4]:
documents = []

for obj in a.objects.all():
    link = f"https://storage.yandexcloud.net/issoproduction/{obj.key}"
    customer, project, name = obj.key.split("/")
    documents.append(
        {
            'link': link,
            'customer_inn': customer,
            'tender_id': project,
            'name': name
        }
    )

In [5]:
documents[:5]

[{'link': 'https://storage.yandexcloud.net/issoproduction/5000001525/0148200005414000386/01__06_1__07__6__08__1-3__09_6-10.pdf',
  'customer_inn': '5000001525',
  'tender_id': '0148200005414000386',
  'name': '01__06_1__07__6__08__1-3__09_6-10.pdf'},
 {'link': 'https://storage.yandexcloud.net/issoproduction/5000001525/0148200005414000386/02_1.pdf',
  'customer_inn': '5000001525',
  'tender_id': '0148200005414000386',
  'name': '02_1.pdf'},
 {'link': 'https://storage.yandexcloud.net/issoproduction/5000001525/0148200005414000386/03_1.pdf',
  'customer_inn': '5000001525',
  'tender_id': '0148200005414000386',
  'name': '03_1.pdf'},
 {'link': 'https://storage.yandexcloud.net/issoproduction/5000001525/0148200005414000386/04_1.pdf',
  'customer_inn': '5000001525',
  'tender_id': '0148200005414000386',
  'name': '04_1.pdf'},
 {'link': 'https://storage.yandexcloud.net/issoproduction/5000001525/0148200005414000386/05_1.pdf',
  'customer_inn': '5000001525',
  'tender_id': '0148200005414000386',


In [6]:
len(documents)

49230

In [7]:
from contextlib import closing

import psycopg2
from psycopg2.extras import DictCursor
from config import DATABASE, USER, PASSWORD, HOST

DATABASE = 'isso_prod'
USER = 'postgres'
PASSWORD = 'ws3iysiw'
HOST = 'localhost'

In [15]:
def insert_into_tendersapp_file(data):
    with closing(psycopg2.connect(dbname=DATABASE, user=USER, password=PASSWORD, host=HOST)) as conn:
        with conn.cursor(cursor_factory=DictCursor) as cursor:
            cursor.execute(
                f"""
                insert into tendersapp_file 
                (
                filename,
                link_to_file,
                customer_inn,
                tender_id)
                values (
                '{data['name']}', 
                '{data['link']}', 
                '{data['customer_inn']}',
                '{data['tender_id']}'
                ) 
                on conflict(id) do nothing;
                """
            )
        conn.commit()

In [16]:
for i in tqdm.tqdm(documents):
    insert_into_tendersapp_file(i)

100%|██████████| 49230/49230 [04:42<00:00, 174.23it/s]
